주식 백테스트

In [3]:
from pykrx import stock
import numpy as np

In [98]:
#
start_year = "2017"
start_date = start_year+"0101"
ma_period = 5

#
end_date = str(int(start_date[:4])+1) + "1231"
target = '1028'
original_df = stock.get_index_ohlcv(start_date, end_date, target)

df = original_df[['종가']]
ma = df['종가'].rolling(ma_period).mean()
df['ma'] = ma.shift(1)

cond = df['종가'] > df['ma']
df['status'] = np.where(cond, 1, 0)
df.iloc[-1, -1] = 0

매수조건 = (df['status']==1) & (df['status'].shift(1)==0)
매도조건 = (df['status']==0) & (df['status'].shift(1)==1)
df['bs'] = np.where(매수조건, 1, 0)
df['ss'] = np.where(매도조건, 1, 0)

slippage = 0.005
수익률 = df.loc[매도조건, '종가'].reset_index(drop=True) / df.loc[매수조건, '종가'].reset_index(drop=True)
수익률.apply(lambda x:x-slippage)
수익률.cumprod()

C:\Users\jchoi\AppData\Local\Temp\ipykernel_38124\103913776.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ma'] = ma.shift(1)
C:\Users\jchoi\AppData\Local\Temp\ipykernel_38124\103913776.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['status'] = np.where(cond, 1, 0)
C:\Users\jchoi\AppData\Local\Temp\ipykernel_38124\103913776.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

0     1.059827
1     1.056145
2     1.052488
3     1.050627
4     1.032580
5     1.035603
6     1.055922
7     1.044435
8     1.035700
9     1.061564
10    1.062413
11    1.055604
12    1.054439
13    1.049331
14    1.038975
15    1.035062
16    1.039119
17    1.034158
18    1.095728
19    1.085391
20    1.091432
21    1.083957
22    1.093378
23    1.084243
24    1.077674
25    1.058364
26    1.120895
27    1.115466
28    1.144639
29    1.130616
30    1.158076
31    1.095536
32    1.125418
33    1.124738
34    1.136835
35    1.149837
36    1.130706
37    1.128311
38    1.120969
39    1.111330
40    1.255684
41    1.250892
42    1.220482
43    1.197483
44    1.202545
45    1.205948
46    1.311665
47    1.284686
48    1.294048
49    1.300058
50    1.330013
51    1.320620
52    1.464417
53    1.543996
54    1.520142
55    1.593709
Name: 종가, dtype: float64

In [97]:
(수익률.mean()-1)*수익률.count()

0.31467866311941606